In [96]:
! pip install scikit-learn

  Using cached scikit_learn-1.3.0-cp39-cp39-win_amd64.whl (9.3 MB)
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/44.1 MB ? eta -:--:

In [97]:
import finpy_tse as fpy
import pandas as pd
import sqlite3 as db
import pytse_client as tse
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

portfollio = 'افران توان اوج پالایش وخارزم اعتلا وبوعلی تمشک گوهران وهامون ومدیر وتوسم وسبحان بوعلی وسکاب مدیریت وصنعت کیان وسپه ومهان'
portfollio_eng = 'vmahan vsepah kian vsanaat modiriat vsakab booali vsobhan vtosam vmodir vhamoon goharan tameshk vbooali alaa vkharazm palayesh owj tavaan afran'
my_pf = portfollio.split()
my_pf_eng = portfollio_eng.split()
my_pf_eng = my_pf_eng[::-1]



## دریافت ریز معاملات

In [26]:
i = 1
print(my_pf_eng[i])
symbol = my_pf[i]
eng = my_pf_eng[i]
symbol

tavaan


'توان'

In [ ]:
symbol = my_pf[i]
eng_symbol = my_pf_eng[i]
df_price_details =fpy.Get_IntradayTrades_History(
                    stock=symbol,
                    start_date='1402-04-13',
                    end_date='1402-04-14',
                    jalali_date=True,
                    combined_datatime=False,
                    show_progress=True)
df_price_details.head()

اضافه کردن ریز معاملات به دیتا بیس

In [ ]:
conn = db.connect('my_bours_price_details.db')
cur = conn.cursor()

df_price_details.reset_index(inplace=True)
df_price_details_new = df_price_details.rename(columns={"J-Date": "Date"})    
df_price_details_new.to_sql(eng_symbol,conn,if_exists='append',index=False)

conn.commit()
cur.close()
conn.close()

## دریافت عمق فقط برای یک نماد

In [ ]:
startDate = '1402-04-10'
endDate = '1402-04-14'
stk = symbol

conn = db.connect('my_bours_omgh.db')
cur = conn.cursor()
df_omgh = fpy.Get_IntradayOB_History(
            stock= stk ,
            start_date= startDate,
            end_date= endDate,
            jalali_date=True,
            combined_datatime=False,
            show_progress=True)
print('done')

df_omgh.reset_index(inplace=True)
df_omgh = df_omgh.rename(columns={"J-Date": "Date"})
df_omgh.to_sql(f'{eng}_omgh_daily',conn,if_exists='append',index=False)

conn.commit()
cur.close()
conn.close()

## دریافت عمق فقط برای چند نماد

In [ ]:
startDate = '1402-01-28'
endDate = '1402-01-30'

conn = db.connect('my_bours_omgh.db')
cur = conn.cursor()


for sym,eng in zip(my_pf,my_pf_eng):
    df_omgh = fpy.Get_IntradayOB_History(
            stock=sym,
            start_date= startDate,
            end_date= endDate,
            jalali_date=True,
            combined_datatime=False,
            show_progress=True)
    
    df_omgh.reset_index(inplace=True)
    df_omgh_new = df_omgh.rename(columns={"J-Date": "Date"})    
    df_omgh_new.to_sql(f'{eng}_omgh_daily',conn,if_exists='append',index=False)
    print(f'{sym} is done')


# close the connection object
conn.commit()
cur.close()
conn.close()

# دریافت قیمت روزانه 

In [30]:
conn = db.connect('daily_price.db')
cur = conn.cursor()

for sym,eng_sym in zip(my_pf,my_pf_eng):
    ticker = tse.Ticker(sym)
    res = tse.download(symbols=sym, write_to_csv=False)
    df = res[sym]
    df.to_sql(f'{eng_sym}_daily',conn,if_exists='append',index=False)
    print(f'{sym} is done')

conn.commit()
cur.close()
conn.close()

افران is done
توان is done
اوج is done
پالایش is done
وخارزم is done
اعتلا is done
وبوعلی is done
تمشک is done
گوهران is done
وهامون is done
ومدیر is done
وتوسم is done
وسبحان is done
بوعلی is done
وسکاب is done
مدیریت is done
وصنعت is done
کیان is done
وسپه is done
ومهان is done


#  بارگزاری دیتاهای پرتفو از دیتا بیس

In [99]:
conn = db.connect('daily_price.db')
chart_data = {}

for com in my_pf_eng:
    com_value = pd.read_sql(f"SELECT * FROM {com}_daily", conn)
    chart_data.update({com : com_value})


conn.close()
chart_data

{'afran':                     date     open     high      low  adjClose          value  \
 0    2020-03-03 00:00:00  10000.0  10000.0  10000.0   10000.0   222233560000   
 1    2020-03-04 00:00:00  10000.0  10000.0  10000.0   10000.0    78939230000   
 2    2020-03-07 00:00:00  10000.0  10000.0  10000.0   10000.0   118910530000   
 3    2020-04-07 00:00:00  10166.0  10200.0  10163.0   10167.0     2658986784   
 4    2020-04-08 00:00:00  10171.0  10171.0  10071.0   10165.0     6627694516   
 ..                   ...      ...      ...      ...       ...            ...   
 779  2023-07-04 00:00:00  21133.0  21134.0  21133.0   21133.0  3620362809535   
 780  2023-07-05 00:00:00  21168.0  21169.0  21168.0   21168.0  3867280673836   
 781  2023-07-08 00:00:00  21189.0  21190.0  21189.0   21189.0  4247959825066   
 782  2023-07-09 00:00:00  21203.0  21204.0  21203.0   21203.0  5429603527001   
 783  2023-07-10 00:00:00  21217.0  21218.0  21217.0   21217.0  2699030136831   
 
         volume  

In [107]:
x = new_df.index
y = new_df['kian']
y2 = new_df['vsakab']

fig, ax = plt.subplots()
ax.plot(x, y)
ax.plot(x, y2)

plt.show()


KeyError: 'vsakab'

In [101]:
df1 = pd.DataFrame({'date': ['2022-01-01', '2022-01-02', '2022-01-03'], 'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'date': ['2022-01-01', '2022-01-03'], 'A': [7, 8], 'B': [9, 10]})
df3 = pd.DataFrame({'date': ['2022-01-02'], 'A': [11], 'B': [12]})

new_df = pd.concat([df1.set_index('date')['A'].rename('afran'),
                    df2.set_index('date')['A'].rename('tavaan'), df3.set_index('date')['A'].rename('kian')], axis=1)

print(new_df)

            afran  tavaan  kian
date                           
2022-01-01      1     7.0   NaN
2022-01-02      2     NaN  11.0
2022-01-03      3     8.0   NaN


# بارگزاری چند تیبل از یک دیتابیس

In [ ]:
conn = db.connect('daily_price.db')

# Execute a SQL query and store the results in a Pandas dataframe
df_tables = pd.read_sql('SELECT * FROM table1 JOIN table2 ON table1.id = table2.id JOIN table3 ON table1.id = table3.id JOIN table4 ON table1.id = table4.id', conn)

# Close the database connection
conn.close()

# بارگزاری چند ستون از یک تیبل

In [22]:
conn = db.connect('daily_price.db')

# Execute a SQL query and store the results in a Pandas dataframe
df_tavan = pd.read_sql('SELECT date, adjClose, volume, count FROM توان_daily_price', conn)

# Close the database connection
conn.close()
df_tavan

,date,adjClose,volume,count
0,2022-11-05 00:00:00,10000.0,490000000,9303
1,2022-11-20 00:00:00,10466.0,24844766,2160
2,2022-11-21 00:00:00,10399.0,27474810,2280
3,2022-11-22 00:00:00,10433.0,14450059,1395
4,2022-11-23 00:00:00,10539.0,9759567,1140
...,...,...,...,...
145,2023-07-03 00:00:00,19587.0,69575887,6729
146,2023-07-04 00:00:00,20120.0,40331724,6741
147,2023-07-05 00:00:00,19874.0,20089639,3825
148,2023-07-08 00:00:00,19770.0,17741855,3815


In [108]:
>>> from sklearn.preprocessing import Normalizer
>>> X = [[4, 1, 2, 2],
...      [1, 3, 9, 3],
...      [5, 7, 5, 1]]
>>> transformer = Normalizer().fit(X)  # fit does nothing.
>>> transformer

Normalizer()

In [109]:
transformer.transform(X)

array([[0.8, 0.2, 0.4, 0.4],
       [0.1, 0.3, 0.9, 0.3],
       [0.5, 0.7, 0.5, 0.1]])